In [33]:
import pandas as pd

videos_per_topic_docs_df = pd.read_csv("videos_per_topic_docs.csv")
video_id_to_doc_mapping_df = pd.read_csv("video_id_to_doc_mapping.csv")

video_id_to_doc_mapping_df


,Video Id,Video Title,Doc Name
0,--8n6A8Q6M0,$200 Luxury Beach Hotel in The Philippines 🇵🇭,0
1,-1B7cVoZr1c,Marine reacts to the Philippine Light Reaction...,1
2,-7vF5F-1btE,Ultimate Filipino Food Festival In The Netherl...,2
3,-9bfDHHneyU,SHOWING MY SISTER SB19 'GENTO' Music Video,3
4,-C5iB25BRsA,10 Reasons/Do not Retire TO the Philippines/Mo...,4
...,...,...,...
2699,zm_8N4vFnZw,PHILIPPINES PROVINCE LIFE IS SO DIFFERENT FROM...,2699
2700,zpT46etTP6E,"Starring real-life gay couple, latest Filipino...",2700
2701,zqvDHfgxWh8,🇰🇷Koreans React to P-pop Idol Group | Lovey Do...,2701
2702,zvTP6wl9sTU,FOREIGNER LIVING IN YHE PHILIPPINES WEEKLY FO...,2702


In [34]:
# videos_per_topic_docs_df = videos_per_topic_docs_df.iloc[1:].reset_index(drop=True)

videos_per_topic_docs_df

,Topic,Custom Topic Label,Video_Ids
0,-1,Topic -1: Outliers,"['178', '358', '495', '685', '704', '822', '89..."
1,0,Topic 0: Delicious Dishes and Dining,"['0', '2', '3', '4', '5', '7', '8', '9', '10',..."
2,1,Topic 1: Learning and Speaking Foreign Language,"['23', '34', '36', '40', '48', '59', '69', '10..."
3,2,Topic 2: Christmas Season & Celebration,"['5', '19', '23', '32', '35', '44', '47', '50'..."
4,3,Topic 3: Experiences of Children in Community ...,"['9', '22', '24', '70', '127', '131', '132', '..."
...,...,...,...
117,116,Topic 116: Chicken & its Specific Related Dishes,"['210', '263', '277', '323', '374', '380', '39..."
118,117,Topic 117: Filipino Phrases,"['18', '29', '84', '148', '282', '353', '407',..."
119,118,Topic 118: Mommy and Baby's Morning Events,"['27', '44', '76', '162', '244', '256', '268',..."
120,119,Topic 119: Common Filipino Dishes,"['26', '323', '367', '436', '599', '695', '766..."


In [35]:
import json
import pandas as pd

# HLTM
topic_map_filepath = './HLTM/output-jsons/' + 'T3.topics.json'

with open(topic_map_filepath, 'r') as f:
    data = json.load(f)

topic_data = []
for entry in data:
    topic_id = entry['topic']
    video_ids = [doc[0] for doc in entry['doc']]  # discard probabilities
    topic_data.append({'topic_id': topic_id, 'video_ids': video_ids})

# Convert to DataFrame
hltm_df = pd.DataFrame(topic_data)

# Display the first few rows
hltm_df.head()




,topic_id,video_ids
0,Z22,"[65, 105, 321, 337, 494, 514, 536, 721, 759, 8..."
1,Z1174,"[5, 19, 21, 25, 41, 43, 52, 54, 71, 76, 86, 88..."
2,Z1197,"[1, 3, 8, 17, 27, 33, 41, 43, 51, 52, 54, 58, ..."
3,Z11,"[811, 2479, 538, 1908, 2643, 887, 1869, 2280, ..."
4,Z232,"[51, 144, 168, 191, 343, 660, 685, 732, 1054, ..."


In [36]:
import pandas as pd

# Ensure video IDs are stored as sets of strings
hltm_df['video_ids'] = hltm_df['video_ids'].apply(lambda x: set(map(str, x)))
videos_per_topic_docs_df['Video_Ids'] = videos_per_topic_docs_df['Video_Ids'].apply(lambda x: set(map(str, x)))

# List to store all matching results
all_matches = []

# Loop over each HLTM topic
for _, hltm_row in hltm_df.iterrows():
    topic_id = hltm_row['topic_id']
    vids_hltm = hltm_row['video_ids']
    
    # Compare with each human-labeled topic
    for _, doc_row in videos_per_topic_docs_df.iterrows():
        topic_num = doc_row['Topic']
        label = doc_row['Custom Topic Label']
        vids_label = doc_row['Video_Ids']
        
        # Compute actual intersection of video IDs
        overlap_ids = vids_hltm & vids_label
        overlap_count = len(overlap_ids)
        
        # Only store if there's any overlap
        if overlap_count > 0:
            all_matches.append({
                'hltm_topic_id': topic_id,
                'matched_topic': topic_num,
                'custom_label': label,
                'overlap_count': overlap_count,
                'overlap_ids': list(overlap_ids),
                'hltm_topic_size': len(vids_hltm),
                'matched_topic_size': len(vids_label),
                'overlap_ratio': overlap_count / min(len(vids_hltm), len(vids_label))
            })

# Create final association DataFrame
association_df = pd.DataFrame(all_matches)

# Sort by overlap count or ratio for better visibility
association_df = association_df.sort_values(
    by=['overlap_ratio', 'overlap_count'], 
    ascending=False
)

# View the top associations
association_df.head()

best_matches_df = association_df.sort_values('overlap_ratio', ascending=False)\
                               .groupby('hltm_topic_id').first().reset_index()

best_matches_df.head()

,hltm_topic_id,matched_topic,custom_label,overlap_count,overlap_ids,hltm_topic_size,matched_topic_size,overlap_ratio
0,Z11,109,Topic 109: Mango Fruit,1,[3],145,15,0.066667
1,Z110,-1,Topic -1: Outliers,4,"[5, 0, 1, 8]",723,15,0.266667
2,Z1100,59,Topic 59: Rainy Season,1,[0],451,15,0.066667
3,Z1101,15,Topic 15: Walking by the Beach Experience,3,"[0, 8, 4]",463,15,0.200000
4,Z1102,18,Topic 18: Sagada Burial Caves & Hanging Coffins,2,"[0, 4]",618,15,0.133333
